In [2]:
from pathlib import Path
from retinaface import RetinaFace
from tensorflow import keras
from keras_vggface.utils import preprocess_input
import numpy as np

In [24]:
import cv2

In [29]:
from keras_vggface.utils import preprocess_input

In [18]:
cwd = Path(os.path.abspath(''))
sys.path.append(str(cwd.parent.parent))
sys.path.append(str(cwd.parent.parent/"recognition"))

In [19]:
sys.path

['/Users/kmielcar/Documents/projects/faces_classification/classification/vgg',
 '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python38.zip',
 '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8',
 '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/lib-dynload',
 '',
 '/Users/kmielcar/Documents/projects/faces_classification/venv/lib/python3.8/site-packages',
 PosixPath('/Users/kmielcar/Documents/projects/faces_classification/classification'),
 PosixPath('/Users/kmielcar/Documents/projects/faces_classification'),
 '/Users/kmielcar/Documents/projects/faces_classification',
 '/Users/kmielcar/Documents/projects/faces_classification',
 '/Users/kmielcar/Documents/projects/faces_classification',
 '/Users/kmielcar/Documents/projects/faces_classification',
 '/Users/kmielcar/Documents/projects/faces_classification/recognition']

In [20]:
from crop import crop

In [35]:
from common import EXPECTED_SIZE, GIRLS

ImportError: cannot import name 'GIRLS' from 'common' (/Users/kmielcar/Documents/projects/faces_classification/common.py)

In [21]:
test_dir = Path("../../images/test/Grafiti/")
output_dir = Path("../../images/results/")

In [22]:
image_paths = list(test_dir.glob("*jpg"))

In [31]:
model = keras.models.load_model('trained_vgg_on_girls/')

In [23]:
test_img = image_paths[0]

In [27]:
img_bgr = cv2.imread(str(test_img))
img_color = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

In [28]:
detected_faces_desc = RetinaFace.detect_faces(str(test_img))

In [40]:
detected_faces_desc

{'face_1': {'score': 0.9995710253715515,
  'facial_area': [1525, 1936, 1782, 2288],
  'landmarks': {'right_eye': [1598.3859, 2070.3555],
   'left_eye': [1717.3215, 2065.498],
   'nose': [1664.9056, 2099.313],
   'mouth_right': [1613.1334, 2183.6394],
   'mouth_left': [1719.8961, 2178.893]}}}

In [44]:
for face_details in detected_faces_desc.values():
    face_roi = face_details['facial_area']
    
    cropped_face_img = crop(img_bgr, *face_roi)
    cropped_face_img = cropped_face_img.astype('float32')
    sample = np.expand_dims(cropped_face_img, axis=0)
    sample = preprocess_input(sample, version=2)
    pred = model.predict(sample)

1/1 [==============================] - 0s 63ms/step


In [45]:
pred

array([[1.7874803e-22, 1.0000000e+00, 1.6545585e-19]], dtype=float32)

In [49]:
ind = np.argmax(pred[0])

In [50]:
proba = pred[0][ind]

In [51]:
text = "{}: {:.2f}%".format("Shira", proba * 100)

In [54]:
left, top, right, bottom = face_roi
y = top - 10 if top - 10 > 10 else top + 10
cv2.rectangle(img_bgr, (left, top), (right, bottom), (0, 0, 255), 2)
marked_img = cv2.putText(img_bgr, text, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

In [55]:
cv2.imwrite("out.jpg", marked_img)

True